<a href="https://colab.research.google.com/github/boss9xy/123/blob/main/B%E1%BA%A3n_sao_c%E1%BB%A7a_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch-cuda
import torch

major_version, minor_version = torch.cuda.get_device_capability()



!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel

import torch

max_seq_length = 2048

dtype = None

load_in_4bit = True

fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit"
]

model, tokenizer = FastLanguageModel.from_pretrained(

    model_name = "unsloth/mistral-7b-bnb-4bit",

    max_seq_length = max_seq_length,

    dtype = dtype,

    load_in_4bit = load_in_4bit,

)



In [ ]:
model = FastLanguageModel.get_peft_model(

    model,

    r = 16,

    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",

                      "gate_proj", "up_proj", "down_proj",],

    lora_alpha = 16,

    lora_dropout = 0,

    bias = "none",

    use_gradient_checkpointing = False,

    random_state = 3407,

    max_seq_length = max_seq_length,

)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("roneneldan/TinyStories", split = "train[:30]")
EOS_TOKEN = tokenizer.eos_token
def formatting_func(example):
  return example["text"] + EOS_TOKEN

In [ ]:
for row in dataset[:2]["text"]:
    print("========================")
    print(row)

In [ ]:
from trl import SFTTrainer

from transformers import TrainingArguments

from transformers.utils import logging

logging.set_verbosity_info()



trainer = SFTTrainer(

    model = model,

    train_dataset = dataset,

    dataset_text_field = "text",

    max_seq_length = max_seq_length,

    packing = True,

    args = TrainingArguments(

        per_device_train_batch_size = 2,

        gradient_accumulation_steps = 4,

        warmup_ratio = 0.05,

        max_grad_norm = 1.0,

        num_train_epochs = 1,

        learning_rate = 2e-5,

        fp16 = not torch.cuda.is_bf16_supported(),

        bf16 = torch.cuda.is_bf16_supported(),

        logging_steps = 1,

        optim = "adamw_8bit",

        weight_decay = 0.1,

        lr_scheduler_type = "linear",

        seed = 3407,

        output_dir = "outputs",

    ),

)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
 [
    "Once upon a time, there was a kingdom, far far away,"
 ]*1, return_tensors="pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True
)

thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()

length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")

    pass
pass